# Funciones de un solo sentido

## Ejercicio 1

Sea $(a_1,\ldots,a_k)$ una secuencia super-creciente de números positivos (la suma de los términos que preceden a $a_i$ es menor que $a_i$, para todo $i$). Elige $n > \sum a_i$, y $u$ un entero positivo tal que $gcd(n, u) = 1$. Define $a_i^* = ua_i \bmod n$. La función mochila (_knapsack_) asociada a $(a_1^*,\ldots,a_k^*)$ es $$f:\mathbb{Z}_2^k \rightarrow \mathbb{N}, f(x_1,\ldots,x_k) = \sum_{i=1}^k x_ia_i^*$$

Implementa esta función y su inversa, tal y como se explica en _P. J. Cameron, Notes on cryptography_. La llave pública es $((a_1,\ldots,a_k), n, u)$.

In [2]:
import numpy as np
import AritmeticaModular as am
from random import randint

"""
    Generate a super-increasing sequence of length k.
    
    A super-increasing sequence has k elements where
    
                    a_i > sum(a_0,...,a_(i - 1))
                    
    An example: (1, 2, 4, 8, 16)
    
    k: length of the message's block
"""
def generate_super_increasing(k):
    
    # generate the first element of the sequence
    sequence = [0]*k
    sequence[0] = randint(1, k)
    
    for i in range(1, k):
        sequence[i] = sum(sequence[:i]) + randint(sequence[i-1], sequence[i-1]*3)
            
    return np.array(sequence, dtype = np.uint32)

generate_super_increasing(8)

array([    4,    14,    55,   198,   666,  1878,  8251, 24129], dtype=uint32)

Con la función *generate_super_increasing* generamos una secuencia súper-creciente de longitud $k$, donde $k$ es el tamaño de bloque a cifrar del mensaje original.

A continuación, tenemos que tomar un número $n$ tal que $n > \sum_{i=0}^k a_i$ y un número $u$ que sea primo relativo con $n$, es decir, que $gcd(n, u) = 1$.

In [3]:
"""
    Generate the keys for the knapsack cipher.
    
    Parameters:
        block_length: length of the message's block
    Return:
        public_key: modified sequence
        private_key: original super-increasing sequence, n and u
"""

def generate_keys(block_length):
    # generate a sequence
    sequence = generate_super_increasing(block_length)
    
    n = sum(sequence) + sequence[randint(0,block_length-1)]*2
    
    found_u = False
    
    while not found_u:
        
        u = randint(1, n)
        
        if am.ext_euclides(n,u)[0] == 1: 
            found_u = True
            
    public_sequence = np.array(list(map(lambda a_i: (u*a_i) % n, sequence)), dtype = np.uint32)
    
    return public_sequence, (sequence, n, u)

key_pub, key = generate_keys(8)
print("Public key:", key_pub)
print("Private key:", key[0], key[1], key[2])

Public key: [157697 177858 167916 185040  20720 190867 173259 119724]
Private key: [    8    22    76   230   999  4274 14250 61596] 204647 45293


Una vez generadas nuestras llaves, podemos pasar a realizar el cifrado del mensaje. Para cifrar el mensaje, al igual que hacíamos en el cifrado en flujo usando el generador de Geffe, tendremos que pasar el string a binario, y para ello, usaremos las funciones *str_to_binlist* y *bin_to_str*. Una vez realizado esto, podremos pasar a a cifrar el mensaje en bloques de tamaño $k$.

In [4]:
from CifradoFlujo import bin_to_str, str_to_binlist

"""
                            Knapsack cypher
    
    This function take a message and the public
    key, and encrypt the message performing:
            ___n
            \                 a_i * e_i
            /__ 0
            
    where a_i are the bits of the message and e_i
    the elements of the public key.
    
    Parameters:
        - message: a str containing the plain message.
        - key_pub: public key
"""
def knapsack_cypher(message, key_pub):
    # take the block size
    block_size = len(key_pub)
    # transform the message to binary list
    binary_message = str_to_binlist(message)
    # check if the message is divisible by the block size
    # if not, add zeros at the end
    while len(binary_message) % block_size != 0:
        binary_message.append(0)
    
    cypher, binary_message = [], np.array(binary_message)
     # cypher text
    for i in range(0,len(binary_message), block_size):
        cypher.append(np.sum(key_pub[binary_message[i:i + block_size] == 1]))
        
    return cypher


encrypted = knapsack_cypher("Lorem ipsum dolor sit amet", key_pub)

print("Encrypted text:", encrypted)

Encrypted text: [389445, 850344, 704073, 656365, 677085, 167916, 486218, 530814, 823797, 841405, 677085, 167916, 536641, 850344, 557361, 850344, 704073, 167916, 823797, 486218, 721681, 167916, 465498, 677085, 656365, 721681]


In [5]:
from AritmeticaModular import inverse
"""
                                        Knapsack decypher
                        
In order to decrypt a message encrypted with this algorithm,
it's necessary to calculate v as u^{-1} mod n using Euclids' 
algorithm.

Then, the message is decrypted:
            _
        b = vb* mod n
            _     ___  
            = v \       e_i a*_i (mod n)
                   /__
            _          ___
            = (uv) \    e_i a_i
                        /__
            _   ___           
            =   \        e_i a_i   (mod n)
                 /__
                 
Parameters:
    - message: list of integers representing the message.
    - key: private key.
"""

def knapsack_decypher(message, key):
    # unfold the private key and calculate 
    # the inverse of u mod n
    p_key, n, u = key
    v =  inverse(u, n)
    # Concatenate list in python it's very simple if you use the operator + and two list. 
    # That is the reason to make decypher_message an empty list
    decypher_message = []
    # start decypher
    for b in message:
        # b_ = vb mod n
        b_, aux = (b * v) % n, 0
        # start the greedy algorithm
        greedy_result = set()
        for i in range(1, len(p_key) + 1):
            if aux + p_key[-i] <= b_:
                aux += p_key[-i]
                greedy_result.add(p_key[-i])
                if aux == b_:
                    break
        # restore and add the message
        decypher_message += list(map(lambda x: 1 if x in greedy_result else 0, p_key))
        # return the plain text
    return bin_to_str(decypher_message)


knapsack_decypher(encrypted, key)

'Lorem ipsum dolor sit amet'

En la función de descifrado haremos uso de las propiedades de las listas en Python y el uso de _set_ para poder buscar los elementos con una mayor velocidad. Al realizar el algoritmo Greedy, iremos insertando los elementos en un _set_, ya que, una vez haya encontrado el algoritmo Greedy la solución, pasaremos a recomponer el bloque de mensaje. Para ello, usaremos una función _map_ que crea una lista de forma que si $x$ se encuentra en el resultado del Greedy, devuelve un 1 y en caso contrario devuelve un 0.

Esto nos devolverá una lista de unos y ceros que podemos concatenar entre sí usando el operador $+$ de las listas. Cuando tengamos el mensaje completo lo reconstruimos a texto plano usando la función *bin_to_str*.

## Ejercicio 2

Sea $p$ un (pseudo-)primo mayor o igual que vuestro número de identidad. Encuentra un elemento primitivo $\alpha$, de $\mathbb{Z}_p^*$ (se puede usar el libro "_Handbook of Applied Cryptography [2.132 (iv)]_"); para facilitar el criterio, es bueno escoger $p$ de forma que $\frac{p - 1}{2}$ sea también primo, y para ello usamos Miller-Rabin). Definimos $$f:\mathbb{Z}_p \rightarrow \mathbb{Z}_p, x\mapsto\alpha^x$$
Calcula el inverso de tu fecha de nacimiento con el formato AAAAMMDD.

__En lo que sigue, $p$ y $q$ son enteros primos, y $n = pq$.__

In [6]:
from os import environ
import AritmeticaModular as am
from random import randint

"""
                                    Search prime
                        
     Search a prime bigger than a number.
     This prime, depending of the parameters will
     have the property that p and (p - 1) // 2 are prime.
     
     Parameters:
         - n: initial number.
         - mid_prime: search a prime that (p - 1) // 2 is prime too.
"""

# search a prime p that (p-1) // 2 is prime too (probably)
def search_prime(n, mid_prime = True):
    p = n if n % 2 != 0 else n + 1
    if mid_prime:
        while not (am.miller_rabin_test(p) and
                   am.miller_rabin_test((p -1)//2)):
                p +=2
    else:
        while not am.miller_rabin_test(p):
            p += 2
    return p

# initial number
p, β = search_prime(int(environ["ID"])), int(environ["BIRTH"])
α = randint(2, p)

"""
                Search a primitive α of Z_p
    
    α will be an primitive of Z_p if and only if:
    
        * α² != 1
        * α^((p-1) / 2) != 1
"""        
def search_primitive_Zp(p):
    α, found = randint(2, p-2), False
    
    while not found:
        
        if am.big_pow(α, (p - 1) // 2, p) != 1 and α**2 != 1:
            found = True
        else:
            α = randint(2, p - 2)
            
    return α

α = search_primitive_Zp(p)
print("Primitive:", α)

Primitive: 217943


Para calcular el inverso del mi fecha de nacimiento módulo el primer primo que surge de mi DNI, tenemos que calcular primero un elemento primitivo del cuerpo $\mathbb{Z}_p$. Para ello, como hemos elegido $p$ de la forma que $p$ es primo y $\frac{p - 1}{2}$ también lo es, podemos definir $p - 1$ como: $$p - 1 = \underbrace{2}_{q_1} \cdot \underbrace{\frac{p - 1}{2}}_{q_2}$$ Entonces, $\alpha$ será un elemento primitivo si y sólo si cumple las siguientes condiciones:
* $\alpha^2\neq 1 \longrightarrow \alpha\neq1$.
* $\alpha^{\frac{p -1}{2}} \neq 1$.

Por lo tanto, buscaremos un $\alpha$ de manera aleatoria que cumpla estas condiciones. Aunque sea aleatorio, esto tardará bastante poco porque la probabilidad de encontrarse un elemento de este tipo es bastante alta, siendo de $\frac{1}{3}$.

Una vez que tenemos esto, vamos a calcular el logaritmo de la solución para obtener la inversa de la imagen de la función.

In [7]:
inv = am.baby_step_giant_step(α, β, p)
print("Inverso:", inv)

Inverso: 8772195


Ahora para comprobar si la solución es correcta, aplicamos la exponencial para comprobar si obtenemos el mismo resultado, donde $\beta$ es la fecha de nacimiento. 

In [8]:
am.big_pow(α, inv, p) == β

True

## Ejercicio 3

Sea $f:\mathbb{Z}_n \rightarrow \mathbb{Z}_n$ la función de Rabin: $f(x) = x^2$. Sea $n = 48478872564493742276963$. Sabemos que $f(12) = 144= f(37659670402359614687722)$. Usando esta información, calcula $p$ y $q$ (mirar la demostración de "_Lecture Notes on Cryptography_", Lemma 2.43.

In [9]:
x, y = 37659670402359614687722, 12
n = 48478872564493742276963

p = am.ext_euclides((x - y), n)[0]
q = n // p

print("p:", p, "q:", q, "correctos:", (p*q)==n)

p: 159497098847 q: 303948303229 correctos: True


Para resolver este ejercicio, he seguido el razonamiento del Lemma 2.43 del libro "_Lecture Notes on Criptography_", donde si conocemos un número y su cuadrado, es muy fácil extraer un divisor no trivial de $n$, teniendo en cuenta que:

1. Escogemos aleatoriamente un $x \in \mathbb{Z}_n^*$.
2. Tomamos un $y$ que sea $y = x^2 \bmod n$.
3. En caso de que $x \equiv y \bmod n $ no sea cierto, el $\gcd(x - y, n)$ es un divisor no trivial de $n$. Por lo tanto, ya habremos factorizado $n$.

Como en este caso ya disponemos de dos $x$ y sus exponenciación, podemos calcular fácilmente $p$ y $q$.

## Ejercicio 4


Elige $a_0$ y $a_1$ dos cuadrados arbitrarios módulo $n$ ($n$ como en el Ejercicio 3). Sea $$h:\mathbb{Z}_2 \times (\mathbb{Z}_n)^*, h(b,x) = x^2a_0^ba_1^{1 - b}$$ Usa la función de Merkle-Damgard para implentar una función resumen tomando $h$ como función de compresión (esta $h$ fue definida por Glodwasser, Micali y Rivest). Los parámetros $a_0, a_1$ y $n$ se hacen públicos (la función debería admitir un parámetro en el que venga especificado el vector inicial).

In [10]:
h = lambda b, x, a0, a1, n: (am.big_pow(x, 2, n)* (a0**b) * a1**(1 - b)) % n

rand1, rand2 = randint(1, p), randint(1, q)
a0, a1 = am.big_pow(rand1, 2, n), am.big_pow(rand2, 2, n)

print("a0 =", a0, "\na1 =", a1)

a0 = 10585264541946572195344 
a1 = 5554474246001310943504


In [11]:
"""
                Merkle-Damgard Hash Function
            
    This function implements the hash method 
    of Merkle-Damgard.
    
    Parameters:
        - original_message: message to hash.
        - vector: initial vector to initialize the hash
        - a0 and a1: values for the hash function
        - n: value of the product of two primes (p and q)
        - f: hash function.
"""

def MerkleDamgard_hash(original_message, vector, a0, a1, n, f):
    
    bin_message, x = str_to_binlist(original_message), vector
    
    for b in bin_message:
        x = f(b, x, a0, a1,n)
        
    return x
    
MerkleDamgard_hash("Lore ipsum dolor sit amet", 987, a0, a1, n, h)

3033820754094728198930

En este ejercicio, cogemos dos elementos aleatorios entre 1 y $p$, y entre 1 y $q$ (donde $n = p\cdot q$) y calculamos el cuadrado de esos números módulo $n$, para obtener dos cuadrados. Tras esto, pasamos a la función de _Merkle-Damgard_ en el que vamos calculando el hash del mensaje gracias a la función $h$. 

Como resultado de la función, devolvemos el último elemento que obtenemos de aplicar la función Hash.

## Ejercicio 5

Sea $p$ el menor primo entero mayor o igual que tu número de identidad, y sea $q$ el primer primo mayor o igual que tu fecha de nacimiento (AAAAMMDD). Selecciona $e$ tal que $gcd(e, (p -1)(q -1)) = 1$. Define la función RSA $$f:\mathbb{Z}_n \rightarrow \mathbb{Z}_n, x \mapsto x^e$$

Calcula el inverso de 1234567890.

In [12]:
p, q = search_prime(int(environ["ID"])), search_prime(int(environ["BIRTH"])) 

φ, e = (p - 1) * (q - 1), randint(1, q)

while am.ext_euclides(e, φ)[0] != 1:
    e = randint(1, q)
    
d = am.inverse(e, φ) % φ

inverse = am.big_pow(1234567890, d, p*q)
print(inverse)
print(am.big_pow(inverse, e, p*q))

358212130875416
1234567890


En este caso, para el cálculo de la preimagen, partimos de que conocemos el valor de $n$, donde $n = p \cdot q$. Esto se conoce como el _módulo del criptosistema_. Al conocer $p$ y $q$, también conoceremos $\varphi(n) = (p-1)\cdot(q -1)$. Conocido esto, podemos calcular el _exponente de cifrado_, que en este caso es $e$.

Con todos estos datos, podemos calcular $d$ que es el exponente de descifrado. El origen de calcular esto, es que RSA cifra el sistema con la función $f$ definida anteriormente, donde cifra elevando un número al _exponente de cifrado_, mientras que el descifrado se calcula elevando el número cifrado a $d$, donde $d$ tiene una relación matemática con $e$ definida de la siguiente manera $$e\cdot d \equiv 1 \bmod \varphi(n)$$

Si despejamos la congruencia, podemos obtener $d$, y realizando la operación $ 123456790^d\bmod n$, obtendremos su preimagen.

## Ejercicio 6


Sea $n=50000000385000000551$, y sabemos que una inversa de $\mathbb{Z}_n \rightarrow \mathbb{Z}_n, x \mapsto x^5$ es $x \mapsto x^{10000000074000000101}$ (esto es, conoces tanto la llave pública como la llave privada de la función RSA). Encuentra $p$ y $q$ usando el método explicado en "_Notes on cryptography" Page 92_. Compara este prodecimiento con el algoritmo de Miller-Rabin y el ejercicio 3.

In [13]:
n, e, d = 50000000385000000551, 5, 10000000074000000101

def broke_RSA(n, e, d):
    # Let de - 1 = 2^a * b and take a random x: 0 < x < n 
    # calculate the gcd(x,n)
    a, b = am.bifactor(d*e - 1)
    x = randint(1, n)

    if am.ext_euclides(x, n)[0] != 1:
        # we found a factor of n
        p, q = x, n//x
    else:
        y = am.big_pow(x, b, n)
        z = y
        while y % n != 1:
            z = y
            y = am.big_pow(y, 2, n)
            if y % n == (n - 1):
                raise ValueError("The algorithm has failed")
        
        p, q = am.ext_euclides(z + 1, n)[0], am.ext_euclides(z - 1, n)[0]
    
    return p, q

p, q = broke_RSA(n, e, d)
p*q == n

True

En este ejercicio, los datos que tenemos son $n, e, d$, lo que supone, como dice el ejercicio, que disponemos de la clave privada del RSA. Para calcular $p$ y $q$, según el método de "_Notes on cryptography_". En este método, tendremos que calcular $a$ y $b$ tal que $2^a, b = d\cdot e - 1$, además de un $x \in_R \mathbb{Z}_n^*$.

Una vez calculados estos valores, comprobamos is $\gcd(x,n) \neq 1$ o no. En caso de que sea distinto de 1, habremos encontrado uno de los factores de $n$, algo que es muy poco probable. En caso contrario, que será lo más común, pasaremos a calcular un $y$ tal que $y = x^b \bmod n$. A su vez, este valor de $y$ debe ser $y \equiv 1 \bmod n$. Si esta condición no se cumple, buscaremos el valor de $y$ y un $z$ haciendo $z = y$ e $y = y^2 \bmod n$. En caso de que $y \bmod n == -1$ el algoritmo falla.

En caso contrario, tendremos que $p = \gcd(z+1,n)$ y $p=\gcd(z-1,n)$.

La diferencia con el ejercicio 3, es que tenemos un problema totalmente distinto, donde en el problema 3 conocemos $n$ y dos valores en los que conocemos sus raíces cuadradas, lo que simplifica mucho el problema. En este caso, la función del RSA es ligeramente distinta a la de Rabin, y los datos completamente distintos, por lo que no podemos extraer $p$ y $q$ tan fácilmente.

## Ejercicio 7

En este ejercicio se pide implementar un sistema de firma digital y verificación de la firma. Se puede elegir entre firma RSA o DSS.

Al igual que antes, debe realizar tres tareas: generación de claves (ejercicios anteriores), generación de firma y verificación de firma.

Para la generación de firma, se le introducirá un mensaje a cifrar (fichero) y el fichero con la clave (privada), y deberá generar una firma, que se guardará en un fichero de texto. 

Puesto que lo que realmente se firma no es el mensaje, sino un resumen del mensaje, hay que generar un resumen de dicho mensaje, Para esto emplearemos la función SHA1 (se pueden añadir otras funciones resumen). Cualquiera de las implementaciónes de esta función que hay en la red puede ser usada. 

Para la verificaión de la firma, se introduce el mensaje (fichero) que se ha firmado, un fichero con la firma (con el mismo formato que el generado en el apartado anterior) y un fichero con la clave (pública). Deberá responder si la firma es o no válida.

In [14]:
from random import getrandbits

In [ ]:
"""
                        RSA Keys Generator
                        
    This function create a pair of RSA's keys 
    with a fixed length. First it search two primes
    p and q, in order to get n.
    
    Then it search and e that gcd(e, φ(n)) = 1, and 
    computes d as e^{-1} mod φ(n)
    
    The private key is n and d, and the 
    public key is n and e.
    
    Parameters:
            - length: length on bits of the key.
"""

def generate_RSA_keys(length = 1024):
    # Compute the primes p and q
    p = search_prime(getrandbits(length//2), False)
    q = search_prime(getrandbits(length//2), False)
    # Compute φ(n), n and a correct e
    φ, n = (p - 1) * (q - 1), p * q
    e = randint(n - φ, φ)
    while am.ext_euclides(e, φ)[0] != 1:
        e += 1
    
    # Compute the private key
    d = am.inverse(e, φ) % φ

    # Write the public key and the private key on files
    with open("RSA_KEY.pub", 'w') as f:
        f.write(str(n) + "\n" + str(e))
    
    with open("RSA_KEY", 'w') as f:
        f.write(str(n) + "\n" + str(d))

        
generate_RSA_keys(2048)

In [ ]:
from hashlib import sha1, sha256, sha512

def get_hash(file, hash_f):
    # compute the hexhash of the hole file
    # and convert to base 10
    if hash_f == 1:
        file_hash = int(sha1(file.read()).hexdigest(), 16)
    elif hash_f == 2:
        file_hash = int(sha256(file.read()).hexdigest(), 16)
    elif hash_f == 3:
        file_hash = int(sha512(file.read()).hexdigest(), 16)
    else:
        raise ValueError("Not a suitable option for digest algorithm.")
        
    return file_hash

"""
                            RSA_sign_document
                    
    This function takes a file and sign it with
    your private RSA key. The function computes
    the digest of the file with sha1, sha256 or
    sha512 and sign this digest with the private key.
    
    Then, the sign is written on a file.
    
    Parameters:
        - file: path to the file.
        - key. path to the file with the private key
        - hash_f: integer option for the digest algorithm
"""

def RSA_sign_document(file, key, hash_f = 1):
    with open(file, 'rb') as f:
        file_hash = get_hash(f, hash_f)
            
    with open(key, 'r') as f:
        n = int(f.readline())
        d = int(f.readline())
    
    # sign the document sign(m) = h(m)^d mod n
    sign = am.big_pow(file_hash, d, n)
    # write the sign
    with open("sign_of_" + file, 'w') as f:
        f.write(str(sign))
    # the return is only to see that the function works well
    return sign

RSA_sign_document("Lore.txt", "RSA_KEY", 3)

In [ ]:
"""
                            RSA_check_sign
                    
    This function takes a file and check if the sign
    is correct with the public key. As in the sign function,
    it function computes the digest of the file
    with sha1, sha256 or sha512, then it computes
    sign^e mod n and check if the hash values
    are equals..
    
    Parameters:
        - sign_document: path to the sign.
        - document: path to the document file.
        - key. path to the file with the private key
        - hash_f: integer option for the digest algorithm
"""

def RSA_check_sign(sign_document, document, public_key, hash_f = 1):
    # get the sign
    with open(sign_document, 'r') as f:
        sign = int(f.read())
    
    # get the public key
    with open(public_key, 'r') as f:
        n = int(f.readline())
        e = int(f.readline())
    
    # get the document's hash
    with open(document, 'rb') as f:
        h_m = get_hash(f, hash_f) % n
        
    sign = am.big_pow(sign, e, n)
    return h_m == sign
    

RSA_check_sign("sign_of_Lore.txt", "Lore.txt", "RSA_KEY.pub", 3)

Como podemos ver, en la función de generación de claves calculamos los primos $p$ y $q$ para generar $n$, y una vez calculados $n$, podemos calcular $e$ (_exponente de encriptado_) tal que $\gcd(e, \varphi(n)) = 1$ y $d$ (_exponente de desencriptado_) que tiene la siguiente forma: $d = e^{-1} \bmod \varphi(n)$.

Una vez calculado esto, ya tenemos nuestra clave privada, que es el par $(n, d)$ y la clave pública $(n, e)$.

Para firmar un documento, he usado la librería _hashlib_ de Python, donde existen implementaciones de distintos algoritmos para hacer un hash. De estos, he cogido los algoritmos ___SHA-1___, ___SHA-256___ y ___SHA-512___ para hacer el hash de los ficheros, siendo la función *get_hash* la que se encarga de devolvernos el hash del texto según el algoritmo que queramos.  Con esto, para firmar el documento, solo tenemos que leer el fichero que queremos firmar, obtener su hash, firmarlo haciendo $sign(m) = h(m)^d \bmod n$ y escribir la firma en un fichero. 

Una vez obtenida la firma, para comprobar que la firma es correcta, el emisor y el receptor deben ponerse de acuerdo para el algoritmo que realiza el hash del mensaje, y una vez establecido, sólo hay que leer el documento que tiene la firma y obtener el hash original haciendo $h(m) = sign^e \bmod n$. Una vez obtenido, hacemos el hash al fichero firmado y comprobamos si el hash que hemos calculado y hash obtenido de la firma coinciden. En caso de que sea así, la firma es correcta y en caso contrario, el fichero no pertenece al propietario que lo firmó.

In [15]:
def generate_DSS_keys(length = 102):
    # search a prime q between 2^159 and 2^160
    q = search_prime(getrandbits(160), False)
    # select a t so that 0 <= t <= 8 
    t = randint(0, 8)
    # and a prime p where 2^(511 + 64t) < p < 2^(512 + 64t)
    # with the property that q divides p - 1
    p = randint(2**159, 2**160)
    if p %2 == 0:
        p + 1
        
    while (p - 1) % q != 0 and not am.miller_rabin_test(p):
        p += 2
    
    # Select a generator α of the unique cyclic group 
    # of order q in Z*_p
    α = 1
    while α == 1:
        g = randint(2, p - 1)
        α = am.big_pow(g, (p - 1) // q, p)
        
    # Select a random integer a such that 1 <= a <= q - 1
    # and compute y = α^a mod p
    a = randint(1, q -1)
    y = am.big_pow(α, a, p)
    # write on files the keys
    with open("DSS_KEY.pub", 'w') as f:
        f.write(str(p) + "\n" + str(q) + "\n" + str(α) + "\n" + str(y))
        
    with open("DSS_KEY", 'w') as f:
        f.write(str(a))
    

generate_DSS_keys()

KeyboardInterrupt: 

In [ ]:
def DSS_sign_document(document, public_key, hash_f = 1):
    # get the public key
    with open(public_key, 'r') as f:
        p, q = int(f.readline()), int(f.readline())
        α, y = int(f.readline()), int(f.readline())
     
   # and the private key 
    with open(private_key, 'r') as f:
        a = int(f.readline())
    
    # get the hash of the document
    with open(document, 'rb') as f:
        h_m = get_hash(f, hash_f)
        
    # select a random secret integer:  0 < k < q
    k = randint(1, q - 1)
    # compute r = (α^k mod p) mod q
    r = am.big_pow(α, k, p) % q
    # compute k^-1 mod q
    inv_k = am.inverse(k, q)
    # sing = k^-1 * {h(m) + ar} mod q
    sign = (inv_k * (h_m + a*r)) % q
    
    with open("sing_of_"+document, 'w') as f:
        f.write(str(r) + "\n" + str(s))

In [17]:
def DSS_check_sign(sign_file, document, public_key, hash_f = 1):
    # get the public key
    with open(public_key, 'r') as f:
        p, q = int(f.readline()), int(f.readline())
        α, y = int(f.readline()), int(f.readline())
        
    # get the sign
    with open(document, 'rb') as f:
        r, s = int(f.readline()), int(f.readline())
        
    # verify the values of r and s    
    if 0 < r < q or 0 < s < q:
        raise ValueError("Bad sign error.")
        
    # compute w = s^-1 mod q and h(m)
    w = am.inverse(s, q)
    
    with open(document, 'rb') as f:
        h_m = get_hash(f, hash_f)
    
    # compute  u1 = w*h(m) mod q and u2 = rw mod q
    u1, u2 = (w*h_m) % q, (r*w) % q
    # compute v = (α^u1 * y^u2 mod p) mod q
    v = (am.big_pow(α, u1, p) * am.big_pow(y, u2, p) % p) % q
    # accept the signature if and only if v = r
    return v == r